In [1]:
import tempfile
import sys
import subprocess
import shutil
from os.path import join
from elasticsearch import Elasticsearch, helpers
from nir.utils import create_filter_query_function, change_bm25_parameters
from utils import *
import pickle


import json

es = Elasticsearch(["http://193.136.175.98:8125"])

index_name = "bioasq_9b"


In [2]:
queries = load_queries("training9b_wDates.json", maps=[("body","query")])

test_8b = []

for i in range(1,6):
    id_sets = set()
    with open(f"yearly_data/8B{i}_golden.json","r") as f:
        for q in json.load(f)["questions"]:
            id_sets.add(q["id"])
            
    test_8b.append(id_sets)

print(sum([len(x) for x in test_8b]))

queries_8b1 = subset_byId(queries, test_8b[0])

queries_8b1, queries_8b1_gs = separate_queries_goldstandard(queries_8b1, additional_keys=["limit_date"])
print(len(queries_8b1))

500
100


In [3]:
import itertools
import random

## SEARCH PARAMETERS

TOP_N = [10,100, 250, 500]
K1 = list(map(lambda x:x/10, range(0,11,1)))
B = list(map(lambda x:x/100, range(0,101,1)))

RANDOM_SEARCH = True

combinations = list(itertools.product(K1,B))

if RANDOM_SEARCH:
    N_Experiments = 500
    selected = random.sample(combinations, N_Experiments)
    print(selected)
else:
    selected = combinations

count = 0
with open("search_bm25_bioasq_9b_batch_01"+".csv", "w") as fOut:
    header = "k1,b,"+",".join(sum([ ["map@"+str(x),"recall@"+str(x)] for x in TOP_N],[]))+"\n"
    fOut.write(header)
    
    for k1,beta in selected:
        
        print(count,"/",len(selected))
        
        results = execute_search(es, queries_8b1, TOP_N[-1], index_name, k1=k1, b=beta)
        results = create_document_run(queries_8b1, results)
        
        metrics_list = map(lambda x:str(x), sum([ list(evaluation(results, queries_8b1_gs, top)) for top in TOP_N],[]))
        line = f"{k1},{beta}," + ",".join(metrics_list)+"\n"
        fOut.write(line)
        fOut.flush()
        count+=1
        
        

[(0.5, 0.51), (0.3, 0.42), (0.7, 0.36), (0.7, 0.75), (0.9, 0.36), (0.3, 0.53), (1.0, 0.8), (0.3, 0.27), (0.7, 0.78), (0.3, 0.2), (0.7, 0.95), (0.0, 0.67), (1.0, 0.07), (0.2, 0.36), (1.0, 0.55), (0.0, 0.42), (0.8, 0.3), (0.0, 0.47), (0.6, 0.32), (0.4, 0.74), (0.7, 0.98), (0.0, 0.25), (0.8, 0.25), (0.9, 0.2), (0.2, 0.75), (0.0, 0.81), (0.2, 0.65), (0.1, 0.44), (0.1, 0.6), (1.0, 0.31), (0.3, 0.87), (0.5, 0.84), (0.5, 0.1), (0.0, 0.31), (0.0, 0.05), (0.8, 0.86), (0.1, 0.32), (0.1, 0.1), (0.7, 0.63), (0.8, 0.09), (1.0, 0.01), (0.2, 0.12), (0.9, 0.41), (0.5, 0.63), (0.8, 0.16), (0.8, 0.82), (0.0, 0.77), (0.7, 0.38), (0.4, 0.69), (0.0, 0.51), (0.4, 0.11), (0.4, 0.4), (0.0, 0.79), (0.9, 0.25), (0.7, 0.08), (0.1, 0.34), (0.5, 0.97), (0.2, 0.31), (0.0, 0.64), (1.0, 0.88), (0.0, 0.91), (0.6, 0.9), (0.0, 0.76), (0.8, 0.42), (0.5, 0.67), (0.2, 0.16), (1.0, 0.48), (0.8, 0.51), (1.0, 0.44), (0.5, 0.2), (1.0, 0.18), (0.9, 0.53), (0.8, 0.61), (0.5, 0.32), (0.2, 0.93), (0.7, 0.45), (0.0, 0.45), (1.0, 0.

37 / 500query: 80
Setting the k1 and b for BM25
38 / 500query: 80
Setting the k1 and b for BM25
39 / 500query: 80
Setting the k1 and b for BM25
40 / 500query: 80
Setting the k1 and b for BM25
41 / 500query: 80
Setting the k1 and b for BM25
42 / 500query: 80
Setting the k1 and b for BM25
43 / 500query: 80
Setting the k1 and b for BM25
44 / 500query: 80
Setting the k1 and b for BM25
45 / 500query: 80
Setting the k1 and b for BM25
46 / 500query: 80
Setting the k1 and b for BM25
47 / 500query: 80
Setting the k1 and b for BM25
48 / 500query: 80
Setting the k1 and b for BM25
49 / 500query: 80
Setting the k1 and b for BM25
50 / 500query: 80
Setting the k1 and b for BM25
51 / 500query: 80
Setting the k1 and b for BM25
52 / 500query: 80
Setting the k1 and b for BM25
53 / 500query: 80
Setting the k1 and b for BM25
54 / 500query: 80
Setting the k1 and b for BM25
55 / 500query: 80
Setting the k1 and b for BM25
56 / 500query: 80
Setting the k1 and b for BM25
57 / 500query: 80
Setting the k1 and b f

KeyboardInterrupt: 

In [4]:
## Visualizing results

import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_01.csv")
df.sort_values('recall@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
91,0.9,0.40,0.344051,0.435528,0.403558,0.706153,0.404921,0.758365,0.405488,0.821070
68,1.0,0.44,0.342657,0.431916,0.403810,0.705439,0.405265,0.761350,0.405802,0.826070
42,0.9,0.41,0.344630,0.438861,0.403075,0.705439,0.404568,0.758365,0.405134,0.821070
109,0.9,0.42,0.346192,0.438861,0.404626,0.705439,0.406110,0.758365,0.406675,0.821070
29,1.0,0.31,0.337836,0.434935,0.398476,0.705403,0.399866,0.755065,0.400438,0.826070
...,...,...,...,...,...,...,...,...,...,...
21,0.0,0.25,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
153,0.0,0.59,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
17,0.0,0.47,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
15,0.0,0.42,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503


In [6]:
df.sort_values('recall@250',ascending=False)


,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
82,0.6,0.40,0.343682,0.439869,0.404215,0.686471,0.405753,0.770131,0.406121,0.809653
104,0.5,0.28,0.333072,0.439766,0.394090,0.673110,0.395938,0.769715,0.396231,0.797398
73,0.5,0.32,0.334318,0.441535,0.394725,0.673776,0.396435,0.769715,0.396731,0.797398
193,0.6,0.30,0.338577,0.440100,0.399533,0.680360,0.401144,0.769298,0.401566,0.819653
18,0.6,0.32,0.340037,0.439869,0.400715,0.680360,0.402324,0.769298,0.402748,0.819653
...,...,...,...,...,...,...,...,...,...,...
110,0.0,0.27,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
111,0.0,0.49,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
58,0.0,0.64,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
46,0.0,0.77,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503


[0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 1.7,
 1.8,
 1.9,
 2.0,
 2.1,
 2.2,
 2.3,
 2.4,
 2.5,
 2.6,
 2.7,
 2.8,
 2.9,
 3.0]